## CAPM in Python

### What is CAPM?
In finance, CAPM is known as the **Capital Asset Pricing Model** and it is widely used for pricing risky securities and generating expected returns given the risk and a "cost of capital". CAPM operates on a number of assumptions about how markets behave and is important mainly because most market participants adopted this framework.

In modern financial theory, two major assumptions are made:
1. Securities markets are very competitive and "**efficient**"
2. These markets are dominated by **rational, risk-averse investors**, who intend to maximize their own return on investment.

The issue is that in practice, investors do not always behave in such ways and there are externalities that can compromise market efficiency. Regardless, it makes sense to base investment decisions assuming that markets are efficient and dominated by rational people because the alternative is entirely unpredictable and irrational.

The CAP Model is represented by this fundamental formula for calculating expected return of an asset given its risk:
<br><br> $ER_i = R_f + \beta_i (ER_m - R_f)$ <br><br>
$ER_i =$ expected return on investment <br>
$R_f =$ risk-free rate <br>
$\beta_i =$ beta of the investment <br>
$(ER_m - R_f) =$ market risk premium <br>

In [6]:
import numpy as np
import pandas as pd
import matplotlib as plt